# Holder Wallet Conformance Flows (v3) - CTWalletCrossPreAuthorised

# 0.0 Initial setup

## 0.1 Setup conformance

In [10]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [11]:
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK

key_did = KeyDid()

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9KbsAHUdzWyJf9FHRNai6L2uBtxvuwUxCgLmGKd7q5qYKY4aZ4Lz4RDtiVVWF4neVx8LCbGh2Br6ZG1yEUqKNSPoKHDpGidv3MogbDLUi3SbR8W6FBJtzDZ578q9xy6vYixTe


## 1.1 Initiate Credential Issuance

In [15]:
from ebsi_wallet.util import parse_query_string_parameters_from_url
from ebsi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type = CredentialTypes.CTWalletCrossPreAuthorised
# qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = "openid-credential-offer://?credential_offer_uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fissuer-mock%2Foffers%2F37298e4a-45fa-44d4-bd63-6363a7ce37b1"
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)

[18:47:02] Credential offer URI:                                                                    ]8;id=852234;file:///tmp/ipykernel_44505/916358673.py\916358673.py]8;;\:]8;id=769999;file:///tmp/ipykernel_44505/916358673.py#16\16]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/37298e4a-45fa-44d4-bd6                
           3-6363a7ce37b1                                                                                          

[18:47:03] Credential offer:                                                                        ]8;id=929888;file:///tmp/ipykernel_44505/916358673.py\916358673.py]8;;\:]8;id=256716;file:///tmp/ipykernel_44505/916358673.py#19\19]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletCrossPreAuthorised'                                                            
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'urn:ietf:params:oauth:grant-type:pre-authorized_code': {                                       
                       'pre-authorized_code':                                                                      
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZ                
           qcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1Bn                
           WW95dHlrVVozZXlxaHQxajlLYnNBSFVkeld5SmY5RkhSTmFpNkwydUJ0eHZ1d1V4Q2dMbUdLZDdxNXFZS1k0YVo0                
           THo0UkR0aVZWV0Y0bmVWeDhMQ2JHaDJCcjZaRzF5RVVxS05TUG9LSERwR2lkdjNNb2diRExVaTNTYlI4VzZGQkp0                
           ekRaNTc4cTl4eTZ2WWl4VGUiLCJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVu                
           dGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNp                
           LmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwi                
           VmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJpYXQiOjE2ODcx                
           ODA2MDEsImV4cCI6MTY4NzE4MDkwMSwiaXNzIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25m                
           b3JtYW5jZS92My9pc3N1ZXItbW9jayIsImF1ZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29u                
           Zm9ybWFuY2UvdjMvYXV0aC1tb2NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQ                
           Z1lveXR5a1VaM2V5cWh0MWo5S2JzQUhVZHpXeUpmOUZIUk5haTZMMnVCdHh2dXdVeENnTG1HS2Q3cTVxWUtZNGFa                
           NEx6NFJEdGlWVldGNG5lVng4TENiR2gy

[18:47:04] Credential issuer configuration:                                                         ]8;id=884005;file:///tmp/ipykernel_44505/916358673.py\916358673.py]8;;\:]8;id=396379;file:///tmp/ipykernel_44505/916358673.py#22\22]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cred                
           ential',                                                                                                
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-                
           mock/credential_deferred',                                                                              
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework': {

[18:47:05] Authorization server configuration:                                                      ]8;id=751299;file:///tmp/ipykernel_44505/916358673.py\916358673.py]8;;\:]8;id=150137;file:///tmp/ipykernel_44505/916358673.py#25\25]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/aut                
           horize',                                                                                                
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Exchange code for access token

In [16]:
from ebsi_wallet.siop_auth.util import exchange_pre_authorized_code_for_access_token

token_uri = auth_server_configuration.token_endpoint
user_pin = "4022"
access_token_response = await exchange_pre_authorized_code_for_access_token(token_uri, user_pin, credential_offer.grants.get("urn:ietf:params:oauth:grant-type:pre-authorized_code").get("pre-authorized_code"))
console.log("Access token response: ", access_token_response)

{'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjIxNzZlZGRlLTA3ODMtNDE5Ni1hM2U4LTI3NDVmODUxZmEyZSIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJjX25vbmNlIjoiMmM1M2YxMjgtMmVjYi00N2M0LTg2YzEtYzZlNzk2MzAyZGMxIiwiY19ub25jZV9leHBpcmVzX2luIjo4NjQwMCwiY2xpZW50X2lkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JzQUhVZHpXeUpmOUZIUk5haTZMMnVCdHh2dXdVeENnTG1HS2Q3cTVxWUtZNGFaNEx6NFJEdGlWVldGNG5lVng4TENiR2gyQnI2WkcxeUVVcUtOU1BvS0hEcEdpZHYzTW9nYkRMVWkzU2JSOFc2RkJKdHpEWjU3OHE5eHk2dllpeFRlIn0sImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2NrIiwiYXVkIjpbImh0dHBzOi8vYXBpLWN

[18:47:17] Access token response:                                                                   ]8;id=597295;file:///tmp/ipykernel_44505/3801089841.py\3801089841.py]8;;\:]8;id=538635;file:///tmp/ipykernel_44505/3801089841.py#6\6]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYW                
           VZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjIxNzZlZGRlLTA3ODMtNDE5Ni1hM2U4LTI3NDV                
           mODUxZmEyZSIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGV                
           udGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnN                
           pLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc1ByZUF1dGhvcmlzZWQiXX1dLCJjX25vbmNlIjo                
           iMmM1M2YxMjgtMmVjYi00N2M0LTg2YzEtYzZlNzk2MzAyZGMxIiwiY19ub25jZV9leHBpcmVzX2luIjo4NjQwMCw                
           iY2xpZW50X2lkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2JzQUhVZHpXeUpmOUZIUk5haTZMMnVCdHh2dXdVeENnTG1HS2Q3cTVxWUtZNGFaNEx6NFJEdGlWVldGNG5lVng                
           4TENiR2gyQnI2WkcxeUVVcUtOU1BvS0hEcEdpZHYzTW9nYkRMVWkzU2JSOFc2RkJKdHpEWjU3OHE5eHk2dllpeFR                
           lIn0sImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2N                
           rIiwiYXVkIjpbImh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvaXNzdWVyLW1                
           vY2siXSwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2JzQUhVZHpXeUpmOUZIUk5haTZMMnVCdHh2dXdVeENnTG1HS2Q3cTVxWUtZNGFaNEx6NFJEdGlWVldGNG5lVng                
           4TENiR2gyQnI2WkcxeUVVcUtOU1BvS0hEcEdpZHYzTW9nYkRMVWkzU2JSOFc2RkJKdHpEWjU3OHE5eHk2dllpeFR                
           lIiwiaWF0IjoxNjg3MTgwNjM3LCJleHAiOjE2ODcyNjcwMzd9._yhRSH07vUddFZfz8qLh_H10wOeEgvdKpZM1b3                
           C7NwWv3n0hazpXyfoEgN1L2QntWznvT9lM84FfLgaLq-ZD7g',                                                      
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUW                
           lGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjBlODgwOWI3LWE3ZDUtNDBlMS1iNjIyLTJjMDY0YmE                
           zYzA0NSIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1                
           tb2NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2JzQUhVZHpXeUpmOUZIUk5haTZMMnVCdHh2dXdVeENnTG1HS2Q3cTVxWUtZNGFaNEx6NFJEdGlWVldGNG5lVng                
           4TENiR2gyQnI2WkcxeUVVcUtOU1BvS0hEcEdpZHYzTW9nYkRMVWkzU2JSOFc2RkJKdHpEWjU3OHE5eHk2dllpeFR                
           lIiwiYXVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J                
           zQUhVZHpXeUpmOUZIUk5haTZMMnVCdHh2dXdVeENnTG1HS2Q3cTVxWUtZNGFaNEx6NFJEdGlWVldGNG5lVng4TEN                
           iR2gyQnI2WkcxeUVVcUtOU1BvS0hEcEdpZHYzTW9nYkRMVWkzU2JSOFc2RkJKdHpEWjU3OHE5eHk2dllpeFRlIiw                
           iaWF0IjoxNjg3MTgwNjM3LCJleHAiOjE2ODcxODA3NTd9.ZSeq7GDks0l9NgqBZokPVMK7mwTOh_hkkXgbpAXFN_                
           i8D-OYAIteY8k9QlaWeT2Ifl0snQqCaLmsRHBT4WO0uw',                                                          
               c_nonce='2c53f128-2ecb-47c4-86c1-c6e796302dc1',                                                     
               c_nonce_expires_in=86400   

## 1.3 Request credential (cross device)

In [17]:
from ebsi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
credential_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
console.log("Credential response: ", credential_response)

[18:47:20] Credential request JWT:                                                                  ]8;id=953621;file:///tmp/ipykernel_44505/2423221428.py\2423221428.py]8;;\:]8;id=768112;file:///tmp/ipykernel_44505/2423221428.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtV                
           WjNleXFodDFqOUtic0FIVWR6V3lKZjlGSFJOYWk2TDJ1QnR4dnV3VXhDZ0xtR0tkN3E1cVlLWTRhWjRMejRSRHRp                
           VlZXRjRuZVZ4OExDYkdoMkJyNlpHMXlFVXFLTlNQb0tIRHBHaWR2M01vZ2JETFVpM1NiUjhXNkZCSnR6RFo1Nzhx                
           OXh5NnZZaXhUZSN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JzQUhVZHpX                
           eUpmOUZIUk5haTZMMnVCdHh2dXdVeENnTG1HS2Q3cTVxWUtZNGFaNEx6NFJEdGlWVldGNG5lVng4TENiR2gyQnI2                
           WkcxeUVVcUtOU1BvS0hEcEdpZHYzTW9nYkRMVWkzU2JSOFc2RkJKdHpEWjU3OHE5eHk2dllpeFRlIiwidHlwIjoi                
           b3BlbmlkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2Nvb                
           mZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg3MjY3MDQwLCJpYXQiOjE2ODcxODA2NDAsImlzcyI6I                
           mRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUtic0FIVWR6V3lKZ                
           jlGSFJOYWk2TDJ1QnR4dnV3VXhDZ0xtR0tkN3E1cVlLWTRhWjRMejRSRHRpVlZXRjRuZVZ4OExDYkdoMkJyNlpHM                
           XlFVXFLTlNQb0tIRHBHaWR2M01vZ2JETFVpM1NiUjhXNkZCSnR6RFo1NzhxOXh5NnZZaXhUZSIsIm5vbmNlIjoiM                
           mM1M2YxMjgtMmVjYi00N2M0LTg2YzEtYzZlNzk2MzAyZGMxIn0.OJKOHdfBFshEkRTwDgzyay-nTBJsrElYIYD3P                
           ZwhdCaOyT4y44KP-zDvT6TNW9QqoQEKHpIxtWH8E03OXduw3Q                                                       

[18:47:21] Credential response:                                                                     ]8;id=510474;file:///tmp/ipykernel_44505/2423221428.py\2423221428.py]8;;\:]8;id=870120;file:///tmp/ipykernel_44505/2423221428.py#6\6]8;;\
           {                                                                                                       
               'format': 'jwt_vc',                                                                                 
               'credential':                                                                                       
           'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWl                
           kI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNv                
           bmZvcm1hbmNlIzljN2E1MDU3LTcwODctNGYxOC04ZWFmLTg0ZWNkZTk2MGY0MyIsInN1YiI6ImRpZDprZXk6ejJk                
           bXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUtic0FIVWR6V3lKZjlGSFJOYWk2TDJ1                
           QnR4dnV3VXhDZ0xtR0tkN3E1cVlLWTRhWjRMejRSRHRpVlZXRjRuZVZ4OExDYkdoMkJyNlpHMXlFVXFLTlNQb0tI                
           RHBHaWR2M01vZ2JETFVpM1NiUjhXNkZCSnR6RFo1NzhxOXh5NnZZaXhUZSIsImlzcyI6ImRpZDplYnNpOnpoSkFS                
           alBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjg3MTgwNjQxLCJleHAiOjE2ODcyNjcwNDEsImlhdCI6MTY4NzE4                
           MDY0MSwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwi                
           aWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlIzljN2E1MDU3LTcwODctNGYxOC04ZWFmLTg0ZWNkZTk2MGY0MyIsInR5                
           cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldENyb3Nz                
           UHJlQXV0aG9yaXNlZCJdLCJpc3N1ZXIiOiJkaWQ6ZWJzaTp6aEpBUmpQTjY5Y0V0Z1B4SGVuMU1pZCIsImlzc3Vh                
           bmNlRGF0ZSI6IjIwMjMtMDYtMTlUMTM6MTc6MjFaIiwiaXNzdWVkIjoiMjAyMy0wNi0xOVQxMzoxNzoyMVoiLCJ2                
           YWxpZEZyb20iOiIyMDIzLTA2LTE5VDEzOjE3OjIxWiIsImV4cGlyYXRpb25EYXRlIjoiMjAyMy0wNi0yMFQxMzox                
           NzoyMVoiLCJjcmVkZW50aWFsU3ViamVjdCI6eyJpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1G                
           WXJXUGdZb3l0eWtVWjNleXFodDFqOUtic0FIVWR6V3lKZjlGSFJOYWk2TDJ1QnR4dnV3VXhDZ0xtR0tkN3E1cVlL                
           WTRhWjRMejRSRHRpVlZXRjRuZVZ4OExDYkdoMkJyNlpHMXlFVXFLTlNQb0tIRHBHaWR2M01vZ2JETFVpM1NiUjhX                
           NkZCSnR6RFo1NzhxOXh5NnZZaXhUZSJ9LCJjcmVkZW50aWFsU2NoZW1hIjp7ImlkIjoiaHR0cHM6Ly9hcGktY29u                
           Zm9ybWFuY2UuZWJzaS5ldS90cnVzdGVkLXNjaGVtYXMtcmVnaXN0cnkvdjIvc2NoZW1hcy96M01nVUZVa2I3MjJ1                
           cTR4M2R2NXlBSm1uTm16REZlSzVVQzh4ODNRb2VMSk0iLCJ0eXBlIjoiRnVsbEpzb25TY2hlbWFWYWxpZGF0b3Iy                
           MDIxIn0sInRlcm1zT2ZVc2UiOnsiaWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L3RydXN0ZWQt                
           aXNzdWVycy1yZWdpc3RyeS92NC9pc3N1ZXJzL2RpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkL2F0dHJp                
           YnV0ZXMvZjFhY2IzNzVmY2ZiZTExYTYwMDNkNWI3MTBiZjM3ZWNiMjc4MmMxMWE0ZTQyODU3MDEzMzc3MzJjMjY2                
           NGQwMCIsInR5cGUiOiJJc3N1YW5jZUNlcnRpZmljYXRlIn19fQ.f3fZ4eq86H0Knt0efR5ozSFzVam0EZCCE8EFi                
           -yUy79qzc9RpF-0uI9v6CDWnVB4RbXanoopwkQkiAKGuZ7QZQ'                                                      
           }                                                                                                       